In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
!pip install tensorflow
!pip install pandas
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

2023-12-18 12:55:39.238317: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 12:55:39.967942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 12:55:39.968160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 12:55:40.084303: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 12:55:40.366038: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 12:55:40.369077: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Initialize CartPole environment
env = gym.make('CartPole-v1')

In [43]:
def envToSamples(env, episodes=1000):
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    variable_name = [""]
    samples = []
    for episode in range(episodes):
        state = env.reset()[0]
        #print(type(state))
        
        while True :#episode_length <= max_episode_length:
            # Exploration-exploitation trade-off
            action = env.action_space.sample()
                
            #print(env.step(action))
            next_state, reward, done, _, _ = env.step(action)
            state = list(state)
            action = [action]
            reward = [reward]
            next_state = list(next_state)
            sample = state + action + reward + next_state
            #print(sample)
            
            # Store experience in replay memory
            #samples.append((state, action, reward, next_state))

            samples.append(sample)
            state = next_state

            if done:
                break
    df = pd.DataFrame(samples)
    '''
    print(df.columns)
    column_mapping = {str(i): f'variable_{i}' for i in range(state_size)}
    print(column_mapping)
    nx_column_mapping = {str(state_size+i+2): f'nx_variable_{i}' for i in range(state_size)}
    df = df.rename(columns={str(state_size): 'action',str(state_size+1): 'reward'})
    df = df.rename(columns=column_mapping)
    df = df.rename(columns=nx_column_mapping)
    '''
    df.columns = [f'variable_{i}' for i in range(state_size)] + ['action','reward'] + [f'nx_variable_{i}' for i in range(state_size)]
    return df

In [49]:
#Function to return Minimum and Maximum value of a state feature
def min_max_states(non_null_values_col, non_null_values_col_nx):
    return min(non_null_values_col.min(),non_null_values_col_nx.min()), max(non_null_values_col.max(),non_null_values_col_nx.max())

In [51]:
def discretiseToBins(bins, df_data):
    state_var = []
    nx_state_var = []
    for i in range(len(bins)):
        col_name = 'variable_' + str(i)
        col_name_nx = 'nx_' + col_name
        non_null_values_col, non_null_values_col_nx = df_data[col_name].dropna(), df_data[col_name_nx].dropna()
        state_var.append(col_name)
        nx_state_var.append(col_name_nx)
        if bins[i] > 1:
            min_value, max_value = min_max_states(non_null_values_col, non_null_values_col_nx)
            bin_size = (max_value - min_value) / bins[i]
            bin = [min_value + i * bin_size for i in range(bins[i] + 1)]
            df_data[col_name] = pd.cut(non_null_values_col, bins=bins[i], labels=False) + 1
            df_data[col_name_nx] = pd.cut(non_null_values_col_nx, bins=bins[i], labels=False) + 1
    df_discrete = pd.DataFrame(df_data['action'])
    df_discrete['current_state'] = df_data.loc[:, state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)
    df_discrete['next_state'] = df_data.loc[:, nx_state_var].apply(lambda row: ', '.join(map(str, row)), axis=1)
    
    #Discretized Sample to MDP
    df_empirical_probs = df_discrete.groupby(['current_state', 'action', 'next_state']).size().reset_index(name='count')
    #print(df_empirical_probs)
    df_empirical_count = df_discrete.groupby(['current_state', 'action']).size().reset_index(name='count_total')
    #print(df_empirical_count)

    df_empirical_probs = df_empirical_probs.merge(df_empirical_count, on=['current_state', 'action'])
    df_empirical_probs['probability'] = df_empirical_probs['count'] / df_empirical_probs['count_total']
    #print(df_empirical_probs)
    
    return df_empirical_probs[['current_state', 'action', 'next_state', 'probability']]

In [45]:
samples = envToSamples(env,100)
print(samples[0:5])

   variable_0  variable_1  variable_2  variable_3  action  reward  \
0   -0.047826   -0.042111   -0.043835   -0.001301       0     1.0   
1   -0.048668   -0.236578   -0.043861    0.277235       0     1.0   
2   -0.053400   -0.431048   -0.038317    0.555768       1     1.0   
3   -0.062020   -0.235409   -0.027201    0.251264       0     1.0   
4   -0.066729   -0.430132   -0.022176    0.535244       0     1.0   

   nx_variable_0  nx_variable_1  nx_variable_2  nx_variable_3  
0      -0.048668      -0.236578      -0.043861       0.277235  
1      -0.053400      -0.431048      -0.038317       0.555768  
2      -0.062020      -0.235409      -0.027201       0.251264  
3      -0.066729      -0.430132      -0.022176       0.535244  
4      -0.075331      -0.624936      -0.011471       0.820858  


In [52]:
discretised_MDP = discretiseToBins([2,2,2,2],samples)
print(discretised_MDP[0:5])

  current_state  action  next_state  probability
0    1, 1, 1, 1       0  1, 1, 1, 1     0.687500
1    1, 1, 1, 1       0  1, 1, 1, 2     0.250000
2    1, 1, 1, 1       0  1, 1, 2, 2     0.062500
3    1, 1, 1, 1       1  1, 1, 1, 1     0.818182
4    1, 1, 1, 1       1  1, 2, 1, 1     0.181818


In [53]:
def get_state_variable_names(environment):
    # Get the observation space of the environment
    observation_space = environment.observation_space

    # Extract variable names from the observation space
    state_variable_names = []
    if isinstance(observation_space, gym.spaces.Dict):
        # If the observation space is a dictionary
        state_variable_names = list(observation_space.spaces.keys())
    elif isinstance(observation_space, gym.spaces.Tuple):
        # If the observation space is a tuple
        state_variable_names = [f"variable_{i+1}" for i in range(len(observation_space))]
    else:
        # If the observation space is a single variable
        state_variable_names = ["state"]

    return state_variable_names

In [55]:
print(get_state_variable_names(env))

['state']
